In [6]:
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# -------------------------- 1. 配置路径（关联前序阶段结果） --------------------------
PRECOMPUTE_DIR = ".\\precomputed_data"          # 阶段1：预计算的股票数据
SELECTION_DIR = ".\\selection_results_optimized"  # 阶段2：优化后的选股结果
BACKTEST_OUTPUT_DIR = ".\\backtest_results_stat"   # 统计回测结果输出目录
Path(BACKTEST_OUTPUT_DIR).mkdir(exist_ok=True, parents=True)

# -------------------------- 2. 读取核心数据（选股结果+股票K线） --------------------------
print("1. 读取核心数据...")
# 2.1 读取优化后的选股结果（处理NaN值）
selection_df = pd.read_csv(
    f"{SELECTION_DIR}/daily_selected_stocks_optimized.csv",
    parse_dates=["date"]
)
selection_df["selected_stocks"] = selection_df["selected_stocks"].fillna("")  # NaN→空字符串
selection_df["selected_stocks"] = selection_df["selected_stocks"].str.strip()  # 去空格

# 2.2 读取股票K线数据（含开盘价、收盘价，用于计算收益）
stock_df = pd.read_csv(
    f"{PRECOMPUTE_DIR}/stock_data_with_indicators.csv",
    parse_dates=["date"],
    usecols=["date", "stock_code", "open", "close", "paused"]  # 仅保留需要的字段
)
# 读取全市场交易日列表（用于定位T+1、T+2日期）
with open(f"{PRECOMPUTE_DIR}/market_days.pkl", "rb") as f:
    market_days = pickle.load(f)
market_days = [pd.to_datetime(day) for day in market_days]  # 统一转为datetime格式

# 2.3 数据预处理：创建“日期+股票代码”的索引（加速查询）
stock_df["date_stock"] = stock_df["date"].astype(str) + "_" + stock_df["stock_code"]
stock_price_dict = {}  # 存储{日期_股票代码: {"open": 开盘价, "close": 收盘价, "paused": 停牌标识}}
for _, row in stock_df.iterrows():
    stock_price_dict[row["date_stock"]] = {
        "open": row["open"],
        "close": row["close"],
        "paused": row["paused"]
    }

# 验证数据
valid_selection_days = len(selection_df[selection_df["selected_stocks"] != ""])
print(f"数据验证：")
print(f" - 选股成功天数（T日）：{valid_selection_days} 天")
print(f" - 回测时间范围：{market_days[0].strftime('%Y-%m-%d')} 至 {market_days[-1].strftime('%Y-%m-%d')}")
print(f" - 股票数据覆盖数：{len(stock_df['stock_code'].unique())} 只")

# -------------------------- 3. 核心回测逻辑（T日选股→T+1买→T+2卖） --------------------------
print("\n2. 执行统计回测（T日选股→T+1买→T+2卖）...")
# 存储每笔交易的详细信息
trade_records = []
# 存储每日增长率（用于后续连乘）
daily_growth_rates = [1.0]  # 初始增长率=1（无收益）
daily_growth_dates = [market_days[0]]  # 初始日期

# 遍历所有选股日（T日）
for _, row in tqdm(selection_df.iterrows(), total=len(selection_df), desc="回测进度"):
    t_date = row["date"]  # T日：选股日
    selected_stocks = row["selected_stocks"]
    
    # 跳过无选股结果的日期
    if selected_stocks == "":
        continue
    
    # 步骤1：定位T+1（买入日）和T+2（卖出日）
    # 先确认T日在交易日列表中的位置
    if t_date not in market_days:
        continue  # T日非交易日，跳过
    t_date_idx = market_days.index(t_date)
    
    # 确保T+1、T+2存在（不超出交易日范围）
    if t_date_idx + 2 >= len(market_days):
        continue  # 无T+2日，无法卖出，跳过
    t1_date = market_days[t_date_idx + 1]  # T+1日：买入日（开盘价买入）
    t2_date = market_days[t_date_idx + 2]  # T+2日：卖出日（收盘价卖出）
    
    # 步骤2：过滤可交易股票（T+1日非停牌+有开盘价，T+2日有收盘价）
    tradable_stocks = []
    selected_stocks_list = selected_stocks.split(",")  # 拆分股票代码
    for stock_code in selected_stocks_list:
        # 检查T+1日数据（买入条件：非停牌+开盘价有效）
        t1_key = f"{t1_date.strftime('%Y-%m-%d')}_{stock_code}"
        t1_data = stock_price_dict.get(t1_key, None)
        if not t1_data or t1_data["paused"] == 1.0 or t1_data["open"] <= 0:
            continue
        
        # 检查T+2日数据（卖出条件：收盘价有效）
        t2_key = f"{t2_date.strftime('%Y-%m-%d')}_{stock_code}"
        t2_data = stock_price_dict.get(t2_key, None)
        if not t2_data or t2_data["close"] <= 0:
            continue
        
        # 符合条件的股票：加入可交易列表
        tradable_stocks.append({
            "stock_code": stock_code,
            "t1_open": t1_data["open"],    # T+1日开盘价（买入价）
            "t2_close": t2_data["close"]   # T+2日收盘价（卖出价）
        })
    
    # 步骤3：计算每笔交易的收益率（无交易则跳过）
    if len(tradable_stocks) == 0:
        continue
    
    # 等权分配资金：每只股票投入相同资金（假设总投入100万，单只仓位=1/股票数）
    total_investment = 1000000.0  # 初始资金（固定100万，不考虑复利再投）
    per_stock_investment = total_investment / len(tradable_stocks)
    
    # 计算单只股票的收益和收益率
    daily_trade_return = 0.0  # 当日所有交易的平均收益率
    for stock in tradable_stocks:
        stock_code = stock["stock_code"]
        buy_price = stock["t1_open"]    # T+1开盘价买入
        sell_price = stock["t2_close"]   # T+2收盘价卖出
        
        # 计算单只股票的收益率（(卖出价-买入价)/买入价）
        stock_return = (sell_price - buy_price) / buy_price
        # 计算单只股票的收益金额（投入资金×收益率）
        stock_profit = per_stock_investment * stock_return
        
        # 记录单只股票的交易信息
        trade_records.append({
            "T日（选股日）": t_date.strftime("%Y-%m-%d"),
            "T+1日（买入日）": t1_date.strftime("%Y-%m-%d"),
            "T+2日（卖出日）": t2_date.strftime("%Y-%m-%d"),
            "股票代码": stock_code,
            "买入价（T+1开盘）": round(buy_price, 2),
            "卖出价（T+2收盘）": round(sell_price, 2),
            "单只收益率": round(stock_return * 100, 2),
            "单只投入资金": round(per_stock_investment, 2),
            "单只收益金额": round(stock_profit, 2)
        })
        
        # 累加当日收益率（等权平均，单只权重=1/股票数）
        daily_trade_return += stock_return / len(tradable_stocks)
    
    # 步骤4：计算当日增长率（按规则：1 + 0.5×每日收益率）
    daily_growth = 1 + 0.5 * daily_trade_return
    daily_growth_rates.append(daily_growth)
    daily_growth_dates.append(t1_date)  # 增长率对应“买入日”（T+1）

# -------------------------- 4. 计算总收益（增长率连乘） --------------------------
print("\n3. 计算总收益与核心指标...")
# 4.1 处理无交易的情况
if len(daily_growth_rates) <= 1:
    print("警告：无有效交易记录，收益为0")
    total_growth = 1.0
    total_return_rate = 0.0
    final_value = 1000000.0
else:
    # 增长率连乘：总增长率 = product(daily_growth_rates[1:])（跳过初始1.0）
    total_growth = np.prod(daily_growth_rates[1:])
    # 总收益率 = (总增长率 - 1) × 100%
    total_return_rate = (total_growth - 1) * 100
    # 最终资金 = 初始资金 × 总增长率
    final_value = 1000000.0 * total_growth

# 4.2 计算交易统计指标
trade_df = pd.DataFrame(trade_records)
total_trades = len(trade_df)  # 总交易次数（每只股票算1次）
win_trades = len(trade_df[trade_df["单只收益金额"] > 0])  # 盈利交易次数
loss_trades = len(trade_df[trade_df["单只收益金额"] < 0])  # 亏损交易次数
win_rate = (win_trades / total_trades * 100) if total_trades > 0 else 0  # 胜率

# 计算平均收益率、总盈利、总亏损
if total_trades > 0:
    avg_stock_return = trade_df["单只收益率"].mean()  # 单只股票平均收益率
    total_profit = trade_df[trade_df["单只收益金额"] > 0]["单只收益金额"].sum()  # 总盈利
    total_loss = abs(trade_df[trade_df["单只收益金额"] < 0]["单只收益金额"].sum())  # 总亏损
    profit_loss_ratio = (total_profit / total_loss) if total_loss > 0 else float("inf")  # 盈亏比
else:
    avg_stock_return = 0.0
    total_profit = 0.0
    total_loss = 0.0
    profit_loss_ratio = 0.0

# -------------------------- 5. 保存结果（交易记录+收益统计） --------------------------
print("4. 保存回测结果...")
# 5.1 保存交易记录（每笔交易的详细信息）
if total_trades > 0:
    trade_df.to_csv(f"{BACKTEST_OUTPUT_DIR}/trade_records.csv", index=False, encoding="utf-8-sig")

# 5.2 保存每日增长率（用于复盘）
growth_df = pd.DataFrame({
    "日期（T+1买入日）": [d.strftime("%Y-%m-%d") for d in daily_growth_dates],
    "当日增长率（1+0.5×每日收益率）": [round(g, 6) for g in daily_growth_rates],
    "累计增长率（连乘）": [round(np.prod(daily_growth_rates[:i+1]), 6) for i in range(len(daily_growth_rates))]
})
growth_df.to_csv(f"{BACKTEST_OUTPUT_DIR}/daily_growth.csv", index=False, encoding="utf-8-sig")

# -------------------------- 6. 输出回测核心结果 --------------------------
print("\n" + "="*60)
print("回测核心结果（纯统计实现）")
print("="*60)
print(f"回测规则：T日选股 → T+1开盘买入 → T+2收盘卖出")
print(f"增长率计算：当日增长率 = 1 + 0.5 × 每日平均收益率")
print(f"总增长率 = 所有交易日增长率连乘")
print("\n" + "-"*60)
print(f"一、收益指标：")
print(f" - 初始资金：1,000,000.00 元")
print(f" - 最终资金：{final_value:,.2f} 元")
print(f" - 总增长率：{total_growth:.6f}")
print(f" - 总收益率：{total_return_rate:.2f}%")
print(f" - 参与交易的买入日数：{len(daily_growth_rates)-1} 天")  # 减去初始1.0

print(f"\n二、交易统计：")
print(f" - 总交易次数：{total_trades} 次（每只股票算1次）")
if total_trades > 0:
    print(f" - 盈利交易次数：{win_trades} 次")
    print(f" - 亏损交易次数：{loss_trades} 次")
    print(f" - 胜率：{win_rate:.2f}%")
    print(f" - 单只股票平均收益率：{avg_stock_return:.2f}%")
    print(f" - 总盈利金额：{total_profit:,.2f} 元")
    print(f" - 总亏损金额：{total_loss:,.2f} 元")
    print(f" - 盈亏比：{profit_loss_ratio:.2f}" if profit_loss_ratio != float("inf") else " - 盈亏比：无（无亏损）")

print(f"\n三、文件输出：")
print(f" - 交易记录：{BACKTEST_OUTPUT_DIR}/trade_records.csv")
print(f" - 每日增长率：{BACKTEST_OUTPUT_DIR}/daily_growth.csv")
print("="*60)

1. 读取核心数据...
数据验证：
 - 选股成功天数（T日）：680 天
 - 回测时间范围：2023-01-03 至 2025-10-27
 - 股票数据覆盖数：5000 只

2. 执行统计回测（T日选股→T+1买→T+2卖）...


回测进度: 100%|██████████| 680/680 [00:00<00:00, 1163.04it/s]


3. 计算总收益与核心指标...
4. 保存回测结果...

回测核心结果（纯统计实现）
回测规则：T日选股 → T+1开盘买入 → T+2收盘卖出
增长率计算：当日增长率 = 1 + 0.5 × 每日平均收益率
总增长率 = 所有交易日增长率连乘

------------------------------------------------------------
一、收益指标：
 - 初始资金：1,000,000.00 元
 - 最终资金：7,272,905.72 元
 - 总增长率：7.272906
 - 总收益率：627.29%
 - 参与交易的买入日数：678 天

二、交易统计：
 - 总交易次数：2697 次（每只股票算1次）
 - 盈利交易次数：1215 次
 - 亏损交易次数：1468 次
 - 胜率：45.05%
 - 单只股票平均收益率：0.70%
 - 总盈利金额：29,784,396.04 元
 - 总亏损金额：25,100,741.06 元
 - 盈亏比：1.19

三、文件输出：
 - 交易记录：.\backtest_results_stat/trade_records.csv
 - 每日增长率：.\backtest_results_stat/daily_growth.csv
